import

In [1]:
# === Cell 1: Imports ===

import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer
import hopsworks
import config


In [2]:
import os
# os.environ["HF_API_KEY"] = ""
os.environ["SILICONFLOW_API_KEY"] = "sk-rrswmweoulftfoccukdcuvyrlvbtblbufvhtvmrivwrcguaz"

In [3]:
# === Cell 2: Connect to Hopsworks ===

from config import HOPSWORKS_API_KEY
# project = hopsworks.login()

project = hopsworks.login(
        # project=HOPSWORKS_PROJECT,
        api_key_value=HOPSWORKS_API_KEY
    )
fs = project.get_feature_store()


2026-01-11 18:32:14,661 INFO: Initializing external client
2026-01-11 18:32:14,661 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-11 18:32:16,330 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286333


In [4]:
# === Cell 3: Load Feature Views ===

metadata_fv = fs.get_feature_view(
    name="paper_metadata_fv_2",
    version=2,
)

chunk_fv = fs.get_feature_view(
    name="paper_chunk_fv_2",
    version=2,
)

metadata_fv.init_serving(1)
chunk_fv.init_serving(1)


In [5]:
# === Cell 4: Load Embedding Model ===
sentence_transformer = SentenceTransformer(
    config.EMBEDDING_MODEL_NAME
)

2026-01-11 18:32:25,994 INFO: Use pytorch device_name: cpu
2026-01-11 18:32:26,000 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [6]:
# === Cell 5: Similarity Search Engine ===

from functions.similarity_search import SimilaritySearchEngine

search_engine = SimilaritySearchEngine(
    embedding_model=sentence_transformer,
    metadata_feature_view=metadata_fv,
    chunk_feature_view=chunk_fv,
)


In [7]:
# === Cell 6: Context Builder ===

from functions.context_builder import ContextBuilder

context_builder = ContextBuilder(
    max_tokens=2000,
    max_chunks=8,
)


In [8]:
# === Cell 7: Prompt Synthesizer ===

from functions.prompt_synthesis import PromptSynthesizer
prompt_synthesizer = PromptSynthesizer()

# from functions.prompt_synthesis_debug import DebugPromptSynthesizer

# prompt_synthesizer = DebugPromptSynthesizer()



In [9]:
# === Cell 8: MCP Dispatcher ===

from functions.mcp_dispatcher import MCPDispatcher

mcp_dispatcher = MCPDispatcher(
    search_engine=search_engine
)


In [10]:
# === Cell 9: Agentic Inference ===

from functions.agent_loop import AgenticInference
from functions.llm_wrapper import LLMWrapper  

# llm = LLMWrapper(
#     model_name_or_path="mistralai/Mistral-7B-v0.1"  
# )# local llm
llm = LLMWrapper(
    model="Qwen/Qwen3-8B",
    base_url="http://api.siliconflow.cn/v1/",
    api_key=os.getenv("SILICONFLOW_API_KEY"),
    temperature=0.2,
    max_tokens=1024,
)

agent = AgenticInference(
    llm=llm,
    search_engine=search_engine,
    context_builder=context_builder,
    prompt_synthesizer=prompt_synthesizer,
    mcp_dispatcher=mcp_dispatcher,
)


In [11]:
# === Cell 10: Run Agent ===

from functions.agent_ui import launch_agent_ui

ui = launch_agent_ui(agent)
ui.launch(inline=True)


* Running on local URL:  http://127.0.0.1:7860
2026-01-11 18:32:32,377 INFO: HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2026-01-11 18:32:32,397 INFO: HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
* To create a public link, set `share=True` in `launch()`.


2026-01-11 18:32:32,805 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2026-01-11 18:33:19,411 WARNING: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

=== CONTEXT BUNDLE ===
{'items': [{'source_id': 'BX8G68KQ#chunk-7', 'paper_id': 'BX8G68KQ', 'content': 'For example, noise can degrade the quality of the features, complicating the diagnosis process [10], [31]. A study by Prasad and Thalluri,. [32] observed that PCG signals possessed high sensitivity and low strength, which rendered them susceptible to noise interference. Therefore, filtering methods are necessary to remove unwanted noise from PCG signals. Generally, analogue (or digital)- based signal processing techniques are utilised to remove the noise [29]. This feature suggests that differential amplifiers and filters should be used to prevent interference [16]. For example, the Butterworth filters are the most prevalent techniques employed in literature for noise suppression, such as low band-pass [2], high band-pass [33], and band-pass filter [16]. A segmentation process involves dividing PCG signals into four segments: S1, systole, S2, and diastole (see Figure 2). This process